## I want to create accurate confidence intervals for synapse samples

Specifically, take a random sample of presynaptic cells from a certain subclass, and ask what percent of its synapses are onto neurons of a certain subclass. The synapses are not independent however, because there are often many synapses between two neurons.

Moreover, the large counts assumption is often violated for the standard error of proportion formula, so the sampling distribution is not normal and has an unkown standard deviation.

In [256]:
import numpy as np
import matplotlib.pyplot as plt
import json

In [249]:
n_ps = 100  # number of values of parameter p to use (between 0 and 1) to find most likely p given p-hat
ps = np.linspace(1/n_ps, 1, n_ps)
p_edges = np.linspace(0, 1, n_ps + 1)

In [285]:
type(np.array([100]))

numpy.ndarray

In [286]:
def get_probs_of_p_hat(sample_size, num_successes, ps=np.linspace(1/100, 1, 100), n_trials=1000):
    """
    sample_size: int
    num_successes: int or np.array of int, 0 <= num_successes <= sample_size
    ps: proportion parameters to use as prior. defaults to uniform prior with 100 samples
    n_trials: number of trials"""
    probs_of_p_hat = np.empty((len(ps), len(num_successes))) if isinstance(num_successes, np.ndarray) else np.empty(ps.shape)
    for i, p in enumerate(ps):
        successes = np.zeros(num_successes.shape) if isinstance(num_successes, np.ndarray) else 0
        for trial in range(n_trials):
            outcome = np.random.binomial(n=sample_size, p=p)
            successes += (outcome == num_successes)
        if isinstance(num_successes, np.ndarray):
            probs_of_p_hat[i, :] = successes / n_trials
        else:
            probs_of_p_hat[i] = successes / n_trials
                                          
    return probs_of_p_hat

In [251]:
def get_CI(x, probs, confidence=0.95):
    """returns the confidence interval of x, where probs[i] == P(x[i] <= x < x[i+1])
    pre: probs sums to 1 and length is one less than len(x)"""
    tail_area = (1 - confidence) / 2
    assert 1 - tail_area / 100 < sum(probs) < 1 + tail_area / 100
    
    lower = 0
    prev_total = 0
    total = probs[lower]
    while total < tail_area:
        prev_total = total
        lower += 1
        total += probs[lower]

    # linearly interpolate the "exact" x
    lowerx = x[lower] + (x[lower + 1] - x[lower]) * (tail_area - prev_total) / (total - prev_total)
    upper = lower
    
    while total < 1 - tail_area:
        prev_total = total
        upper += 1
        total += probs[upper]
    
    upperx = x[upper] + (x[upper + 1] - x[upper]) * (1 - tail_area - prev_total) / (total - prev_total)
    return lowerx, upperx 

In [296]:
for sample_size in tqdm(range(60, 200)):
    print(sample_size)
    CIs[int(sample_size)] = dict()
    
    num_successes = np.arange(0, sample_size + 1)
    probs_of_p_hat = get_probs_of_p_hat(sample_size, num_successes, ps=ps, n_trials=10_000)
    # because of uniform prior, the probability of p_hat given p, is the probability of p given p_hat after normalization
    probs_of_p = probs_of_p_hat / np.sum(probs_of_p_hat, axis=0)
    for j, nsuc in enumerate(num_successes):
        CI = get_CI(p_edges, probs_of_p[:, j], confidence=0.95)
        CIs[int(sample_size)][int(nsuc)] = (float(CI[0]), float(CI[1]))

  0%|                                                                                          | 0/140 [00:00<?, ?it/s]

60


  1%|▌                                                                                 | 1/140 [00:06<15:40,  6.76s/it]

61


  1%|█▏                                                                                | 2/140 [00:13<15:34,  6.77s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 138/140 [17:11<00:15,  7.84s/it]

198


 99%|███████████████████████████████████████████████████████████████████████████████▍| 139/140 [17:19<00:07,  7.96s/it]

199


100%|████████████████████████████████████████████████████████████████████████████████| 140/140 [17:27<00:00,  7.48s/it]


In [316]:
# for ki in list(CIs.keys()):
#     for k in list(CIs[ki].keys()):
#         temp = CIs[ki][k]
#         del CIs[ki][k]
#         CIs[ki][int(k)] = (float(temp[0]), float(temp[1]))

In [317]:
# type(list(CIs[100].keys())[0])

int

In [318]:
with open("..\data\CIs.json", "w") as f:
    f.write(json.dumps(CIs))

In [280]:
with open("..\data\CIs.json") as f:
    CIs = json.loads(f.read())

In [289]:
probs_of_p.shape

(100, 51)

In [293]:
from tqdm import tqdm

In [149]:
p_hat = num_successes / sample_size
se = np.sqrt(p_hat * (1 - p_hat) / sample_size)
p_hat - 2 * se, p_hat + 2 * se

(-0.1577708763999664, 0.5577708763999665)

In [143]:
np.average(ps, weights=prob_of_p_hat)

0.21232822540781018

In [86]:
prob_of_p_hat

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.004,
       0.01 , 0.014, 0.042, 0.034, 0.046, 0.076, 0.105, 0.104, 0.13 ,
       0.128, 0.123, 0.127, 0.144, 0.14 , 0.111, 0.107, 0.094, 0.084,
       0.081, 0.073, 0.057, 0.047, 0.032, 0.03 , 0.013, 0.02 , 0.005,
       0.002, 0.002, 0.005, 0.004, 0.002, 0.002, 0.   , 0.   , 0.   ,
       0.001, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   ])